In [1]:
!pip uninstall numpy

Found existing installation: numpy 1.14.6
Uninstalling numpy-1.14.6:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.7/dist-packages/numpy-1.14.6.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.14.6


In [2]:
!pip install numpy

     |████████████████████████████████| 15.7 MB 393 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
mxnet 1.4.0 requires numpy<1.15.0,>=1.8.2, but you have numpy 1.21.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
bert-embedding 1.0.1 requires numpy==1.14.6, but you have numpy 1.21.6 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
!pip install bert-embedding

     |████████████████████████████████| 29.6 MB 2.7 MB/s 
     |████████████████████████████████| 209 kB 61.0 MB/s 
     |████████████████████████████████| 13.8 MB 43.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=e9a54676201bb44348951230533c8838afcd76d94abc7825a8331f91ab97099e
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly

In [3]:
!pip install rouge_score

In [1]:
import glob
import re
import pandas as pd


class DataReader:
    def __init__(self, path):
        self.path = path
        self.documents = {}
        self.summaries = {}

    def load_dataset(self):
        document_files = sorted(glob.glob(self.path + "/document*.txt"))
        summary_files = sorted(glob.glob(self.path + "/summary*.txt"))
        for document_file in document_files:
            doc_num = re.findall(r'\d+', document_file)[0]
            with open(document_file, 'r') as file:
                data = file.read().replace('\n', '')
            self.documents[doc_num] = data
        for summary_file in summary_files:
            doc_num = re.findall(r'\d+', summary_file)[0]
            with open(summary_file, 'r') as file:
                data = file.read().replace('\n', '')
            self.summaries[doc_num] = data
        return self.documents, self.summaries

    def load_dataset_from_csv(self):
        df = pd.read_csv(self.path,nrows = 500)
        return df[df.columns[1]].to_list(), df[df.columns[2]].to_list()

In [ ]:
!pip install nltk

In [3]:
#Shortcuts simplify My Drive … 
#In the coming weeks, items in more than one folder will be replaced by shortcuts. Access to files and folders won't change.Learn more
import string

import numpy as np
import nltk
#from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize as nlkt_sent_tokenize
from nltk.tokenize import word_tokenize as nlkt_word_tokenize
from rouge_score import rouge_scorer
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from bert_embedding import BertEmbedding
import csv

nltk.download('stopwords')
nltk.download('punkt')
#from reader import DataReader


# Calculates cosine similarity
def similarity(v1, v2):
    score = 0.0
    if np.count_nonzero(v1) != 0 and np.count_nonzero(v2) != 0:
        score = ((1 - cosine(v1, v2)) + 1) / 2
    return score


def sent_tokenize(text):
    sents = nlkt_sent_tokenize(text)
    sents_filtered = []
    for s in sents:
        sents_filtered.append(s)
    return sents_filtered


def cleanup_sentences(text):
    stop_words = set(stopwords.words('english'))
    sentences = sent_tokenize(text)
    sentences_cleaned = []
    for sent in sentences:
        words = nlkt_word_tokenize(sent)
        words = [w for w in words if w not in string.punctuation]
        words = [w for w in words if not w.lower() in stop_words]
        words = [w.lower() for w in words]
        sentences_cleaned.append(" ".join(words))
    return sentences_cleaned


def get_tf_idf(sentences):
    vectorizer = CountVectorizer()
    sent_word_matrix = vectorizer.fit_transform(sentences)

    transformer = TfidfTransformer(norm=None, sublinear_tf=False, smooth_idf=False)
    tfidf = transformer.fit_transform(sent_word_matrix)
    tfidf = tfidf.toarray()

    centroid_vector = tfidf.sum(0)
    centroid_vector = np.divide(centroid_vector, centroid_vector.max())

    feature_names = vectorizer.get_feature_names_out()

    relevant_vector_indices = np.where(centroid_vector > 0.3)[0]

    word_list = list(np.array(feature_names)[relevant_vector_indices])
    return word_list


def word_vectors_cache_bert(clean_sentences, result):
    word_vectors = dict()
    for i in range(0, len(clean_sentences)):
      for j in range(0, len(result[i][0])):
        word_vectors.update({result[i][0][j]: result[i][1][j]})
    return word_vectors

def build_embedding_representation_bert(words, word_vectors, result):
    embedding_representation = np.zeros(len(result[0][1][0]), dtype="float32")
    word_vectors_keys = set(word_vectors.keys())
    #print(word_vectors_keys)
    count = 0
    for w in words:
        if w in word_vectors_keys:
            embedding_representation = embedding_representation + word_vectors[w]
            count += 1
    if count != 0:
       embedding_representation = np.divide(embedding_representation, count)
    return embedding_representation

def summarize(text, emdedding_model):
    raw_sentences = sent_tokenize(text)
    clean_sentences = cleanup_sentences(text)
    centroid_words = get_tf_idf(clean_sentences)
    word_vectors = word_vectors_cache_bert(clean_sentences, emdedding_model)
    # Centroid embedding representation
    centroid_vector = build_embedding_representation_bert(centroid_words, word_vectors, emdedding_model)
    sentences_scores = []
    for i in range(len(clean_sentences)):
        scores = []
        words = clean_sentences[i].split()

        # Sentence embedding representation
        sentence_vector = build_embedding_representation_bert(words, word_vectors, emdedding_model)

        # Cosine similarity between sentence embedding and centroid embedding
        score = similarity(sentence_vector, centroid_vector)
        sentences_scores.append((i, raw_sentences[i], score, sentence_vector))
    sentence_scores_sort = sorted(sentences_scores, key=lambda el: el[2], reverse=True)
    # for s in sentence_scores_sort:
    #     print(s[0], s[1], s[2])
    count = 0
    sentences_summary = []
    # Handle redundancy
    for s in sentence_scores_sort:
        if count > 100:
            break
        include_flag = True
        for ps in sentences_summary:
            sim = similarity(s[3], ps[3])
            if sim > 0.95:
                include_flag = False
        if include_flag:
            sentences_summary.append(s)
            count += len(s[1].split())

        sentences_summary = sorted(sentences_summary, key=lambda el: el[0], reverse=False)

    summary = "\n".join([s[1] for s in sentences_summary])
    return summary


if __name__ == '__main__':
    reader = DataReader('/content/drive/MyDrive/NLP_Project/cnn_data.csv')
    documents,summaries = reader.load_dataset_from_csv();
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)    
    rouge_1_recalls = np.zeros(shape=(len(documents)))
    rouge_2_recalls = np.zeros(shape=(len(documents)))
    rouge_L_recalls = np.zeros(shape=(len(documents)))
    count = 0
    bert_embedding = BertEmbedding()
    summaries_out = []
    for doc_num in range(len(documents)):
      try:
        count+=1
        print((count/1000)*100, "%")
        document = documents[doc_num]
        summary = summaries[doc_num]
        clean_sentences = cleanup_sentences(document)
        words = []
        for sent in clean_sentences:
            words.append(nlkt_word_tokenize(sent))
        if len(words) > 0:
            model = bert_embedding(clean_sentences)
            generated_summary = summarize(document, model)
            summaries_out.append(generated_summary)
            scores = scorer.score(summary, generated_summary)
            rouge_1 = scores['rouge1'].recall
            rouge_2 = scores['rouge2'].recall
            rouge_L = scores['rougeL'].recall
            rouge_1_recalls[int(doc_num)] = rouge_1
            rouge_2_recalls[int(doc_num)] = rouge_2
            rouge_L_recalls[int(doc_num)] = rouge_L 
      except Exception:
          pass         

    print('Avg rouge 1 score {}'.format(np.average(rouge_1_recalls)))
    print('Avg rouge 2 score {}'.format(np.average(rouge_2_recalls)))
    print('Avg rouge L score {}'.format(np.average(rouge_L_recalls)))

    df = pd.DataFrame(summaries_out)
    df.to_csv("/content/drive/MyDrive/NLP_Project/bert_output_500.csv")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocab file is not found. Downloading.
0.1 %
0.2 %
0.3 %
0.4 %
0.5 %
0.6 %
0.7000000000000001 %
0.8 %
0.8999999999999999 %
1.0 %
1.0999999999999999 %
1.2 %
1.3 %
1.4000000000000001 %
1.5 %
1.6 %
1.7000000000000002 %
1.7999999999999998 %
1.9 %
2.0 %
2.1 %
2.1999999999999997 %
2.3 %
2.4 %
2.5 %
2.6 %
2.7 %
2.8000000000000003 %
2.9000000000000004 %
3.0 %
3.1 %
3.2 %
3.3000000000000003 %
3.4000000000000004 %
3.5000000000000004 %
3.5999999999999996 %
3.6999999999999997 %
3.8 %
3.9 %
4.0 %
4.1000000000000005 %
4.2 %
4.3 %
4.3999999999999995 %
4.5 %
4.6 %
4.7 %
4.8 %
4.9 %
5.0 %
5.1 %
5.2 %
5.3 %
5.4 %
5.5 %
5.6000000000000005 %
5.7 %
5.800000000000001 %
5.8999999999999995 %
6.0 %
6.1 %
6.2 %
6.3 %
6.4 %
6.5 %
6.6000000000000005 %
6.7 %
6.800000000000001 %
6.9 %
7.000

In [7]:
bert_embedding = BertEmbedding()

In [9]:
bert_embedding 